I dati utilizzati in questo notebook sono stati presi dalla competizione di Analytics Vidhya [Practice Problem: Big Mart Sales III](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/#data_dictionary).

# Analisi esplorativa e preprocessamento dei dati

## Indice

1. [Big Mart Sales](#big_mart_sales)<br>
    1.1 [Descrizione](#descrizione)<br>
    1.2 [Leggere i dati e separare la variabile risposta](#leggere_dati)<br>
2. [Analisi esplorativa: studiare le variabili esplicative](#studiare_esplicative)<br>
    2.1 [Contare i valori mancanti](#contare_valori_mancanti)<br>
    2.2 [Dividere le variabili quantitative dalle variabili qualitative](#dividere_quantitative_qualitative)<br>
    2.3 [Variabili quantitative](#quantitative)<br>
    2.4 [Variabili qualitative](#qualitative)<br>
3. [Preprocessare i dati](#preprocessare_dati)<br>
    3.1 [Riempire i valori mancanti](#riempire_valori_mancanti)<br>
    3.2 [Aggregare i livelli simili delle variabili qualitative](#aggregare_livelli_qualitative)<br>
    3.3 [Eliminare le colonne che non si intendono utilizzare](#eliminare_colonne)<br>
4. [Ottenere gli indici degli insiemi di *training*, *validation* e *test*](#train_val_test)<br>
5. [Analisi esplorativa: studiare la relazione tra variabili esplicative e variabile risposta](#studiare_esplicative_risposta)<br>
    5.1 [Relazione tra quantitative e risposta](#quantitative_risposta)<br>
    5.2 [Relazione tra qualitative e risposta](#qualitative_risposta)<br>
    5.3 [Relazione tra esplicative e risposta](#esplicative_risposta)<br>
6. [Trasformare le variabili qualitative in dummy](#dummy)<br>
7. [Standardizzare i dati](#standardizzare)<br>
8. [Pipeline di preprocessamento](#pipeline)<br>
    8.1 [Definire una `Pipeline`](#definire_pipeline)<br>
    8.2 [Preprocessare i dati attraverso la pipeline](#preprocessare_pipeline)<br>
9. [Salvare](#salvare)<br>
    9.1 [Salvare i dati preprocessati](#salvare_dati)<br>
    9.2 [Salvare la pipeline](#salvare_pipeline)<br>

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# 1. Big Mart Sales <a id=big_mart_sales> </a>

## 1.1 Descrizione <a id=descrizione> </a>

### Problem Statement

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.

 

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.

### Data

We have train (8523) and test (5681) data set, train data set has both input and output variable(s). You need to predict the sales for test data set.

|**Variable**                 | Description                                              |
|-----------------------------|----------------------------------------------------------|
|**Item_Identifier**          | Unique product ID                                        |
|**Item_Weight**              | Weight of product                                        |
|**Item_Fat_Content**         | Whether the product is low fat or not                    |
|**Item_Visibility**          | The % of total display area of all products in a store allocated<br/>to the particular product|
|**Item_Type**                |The category to which the product belongs                 |
|**Item_MRP**                 |Maximum Retail Price (list price) of the product          |
|**Outlet_Identifier**        |Unique store ID                                           |
|**Outlet_Establishment_Year**|The year in which store was established                   |
|**Outlet_Size**              |The size of the store in terms of ground area covered     |
|**Outlet_Location_Type**     |The type of city in which the store is located            |
|**Outlet_Type**              |Whether the outlet is just a grocery store or some sort of<br/>supermarket|
|**Item_Outlet_Sales**        |Sales of the product in the particulat store. This is the outcome variable<br/>to be predicted|

## 1.2 Leggere i dati e separare la variabile risposta <a id=leggere_dati> </a>

### Leggere i dati

In [ ]:
PATH = "datasets/big_mart_sales" # cambiare in base a dove si è salvato il dataset

dati = pd.read_csv(PATH + "/Train_UWu5bXk.csv")
print("Dimensione del dataset: {} x {}".format(*dati.shape))
dati.head()

### Dividere le variabili esplicative dalla variabile risposta

In [ ]:
risposta = "Item_Outlet_Sales"
esplicative = sorted(col for col in dati.columns if col != risposta)

X, y = dati[esplicative].copy(), dati[risposta].copy()

# 2. Analisi esplorativa: studiare le variabili esplicative <a id=studiare_esplicative> </a>

## 2.1 Contare i valori mancanti <a id=contare_valori_mancanti> </a>

In [ ]:
dati.isnull().sum()

## 2.2 Dividere le variabili quantitative dalle variabili qualitative <a id=dividere_quantitative_qualitative> </a>

### Controllare i `dtypes` delle colonne

In [ ]:
X.dtypes

### Salvare i nomi delle colonne in due liste distinte

In [ ]:
quantitative = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
qualitative = X.select_dtypes(include=["object"]).columns.tolist()

## 2.3 Variabili quantitative <a id=quantitative> </a>

In [ ]:
X[quantitative].head()

### Descrivere le variabili

In [ ]:
X.describe() # nota: vengono automaticamente considerate solo le colonne numeriche

In [ ]:
plt.figure(figsize=(10, 10))

sns.pairplot(X[quantitative])

plt.show()

## 2.4 Variabili qualitative <a id=qualitative> </a>

In [ ]:
X[qualitative].head()

### Contare il numero di livelli

In [ ]:
X[qualitative].nunique()

### Contare il numero di osservazioni per ogni livello

In [ ]:
for col in qualitative:
    display(X[col].value_counts().head(16)) # mi limito ai primi 16 valori di ogni livello

### Esercizio

Esplorare ulteriormente per via grafica le variabili esplicative (istogrammi, boxplot, ...).

> Suggerimento: considerare le librerie [Matplotlib](https://matplotlib.org/), [Seaborn](https://seaborn.pydata.org/) o, per grafici interattivi, [Bokeh](https://bokeh.pydata.org/en/latest/).

### Esercizio

Elencare quanto scoperto grazie all'analisi esplorativa.

# 3. Preprocessare i dati <a id=preprocessare_dati> </a>

## 3.1 Riempire i valori mancanti <a id=riempire_valori_mancanti> </a>

### Studiare la relazione tra *Item_Identifier* e *Item_Weight*

In [ ]:
weight_grby_id = X[["Item_Identifier", "Item_Weight"]].groupby("Item_Identifier").\
    agg(["count", "min", "max"])["Item_Weight"]
weight_grby_id.sort_values("count", inplace=True, ascending=False)

print("Item_Identifier senza nemmeno un Item_Weight associato: {}".format((weight_grby_id["count"] == 0).sum()))
weight_grby_id = weight_grby_id.loc[weight_grby_id["count"] > 0]
print("Percentuale di Item_Identifier con almeno un Item_Weight per cui min è uguale a max: {}%"
      .format(100 * (weight_grby_id["min"] == weight_grby_id["min"]).mean()))
display(weight_grby_id.head())

### Riempire i valori mancanti di *Item_Weight*

In [ ]:
from msbd.preprocessamento import RiempireNAItemWeight

print(inspect.getsource(RiempireNAItemWeight))

In [ ]:
print("Valori mancanti di Item_Weight prima della sostituzione: {}".format(X["Item_Weight"].isnull().sum()))

riempire_na_item_weight = RiempireNAItemWeight()

X = riempire_na_item_weight.fit_transform(X)

print("Valori mancanti di Item_Weight dopo la sostituzione: {}".format(X["Item_Weight"].isnull().sum()))

### Studiare la relazione tra *Outlet_Location_Type* e *Outlet_Size*

In [ ]:
size_grby_location = X.groupby("Outlet_Location_Type")["Outlet_Size"].value_counts().unstack().fillna(0)

size_grby_location

### Studiare la relazione tra *Outlet_Type* e *Outlet_Size*

In [ ]:
size_grby_type = X.groupby("Outlet_Type")["Outlet_Size"].value_counts().unstack().fillna(0)

size_grby_type

### Riempire i valori mancanti di *Outlet_Size*

In [ ]:
from msbd.preprocessamento import RiempireNAOutletSize

print(inspect.getsource(RiempireNAOutletSize))

### Esercizio

Completare il metodo `transform()`della classe `RiempireNAOutletSize` basandosi sui risultati dell'analisi esplorativa.

In [ ]:
print("Valori mancanti di Outlet_Size prima della sostituzione: {}".format(X["Outlet_Size"].isnull().sum()))

riempire_na_outlet_size = RiempireNAOutletSize()

X = riempire_na_outlet_size.fit_transform(X)

print("Valori mancanti di Outlet_Size dopo della sostituzione: {}".format(X["Outlet_Size"].isnull().sum()))

### Riempire gli ultimi valori mancanti rimasti delle variabili quantitative utilizzando il valor medio

In [ ]:
from msbd.preprocessamento import RiempireNAMedia

print(inspect.getsource(RiempireNAMedia))

In [ ]:
print("Valori mancanti prima della sostituzione: \n{}".format(X.isnull().sum()))

riempire_na_media = RiempireNAMedia()

X = riempire_na_media.fit_transform(X)

print("\nValori mancanti dopo la sostituzione: \n{}".format(X.isnull().sum()))

## 3.2 Aggregare i livelli simili delle variabili qualitative <a id=aggregare_livelli_qualitative> </a>

### Aggregare i livelli simili di *Item_Fat_Content*

In [ ]:
from msbd.preprocessamento import Sostituire

print(inspect.getsource(Sostituire))

In [ ]:
to_replace = {"Item_Fat_Content": {"Low Fat": "Low_Fat", "LF": "Low_Fat", "low fat": "Low_Fat", "reg": "Regular"}}

In [ ]:
print("Livelli prima della sostituzione: {}".format(X["Item_Fat_Content"].unique()))

sostituire_item_fat_content = Sostituire(to_replace)

X = sostituire_item_fat_content.fit_transform(X)

print("Livelli dopo la sostituzione: {}".format(X["Item_Fat_Content"].unique()))

## 3.3 Eliminare le colonne che non si intendono utilizzare <a id=eliminare_colonne> </a>

### Eliminare *Item_Identifier*

In [ ]:
from msbd.preprocessamento import Eliminare

print(inspect.getsource(Eliminare))

In [ ]:
elim = Eliminare("Item_Identifier")

X = elim.fit_transform(X)

esplicative.remove("Item_Identifier")
qualitative.remove("Item_Identifier")

# 4. Ottenere gli indici degli insiemi di *training*, *validation* e *test* <a id=train_val_test> </a>

In [ ]:
from sklearn.model_selection import train_test_split

idx_train, idx_test = train_test_split(y.index.values, test_size=1000, random_state=42)
idx_train, idx_val = train_test_split(idx_train, test_size=1000, random_state=42)

print("Dimensione del training set: {}".format(len(idx_train)))
print("Dimensione del validation set: {}".format(len(idx_val)))
print("Dimensione del test set: {}".format(len(idx_test)))

# 5. Analisi esplorativa: studiare la relazione tra variabili esplicative e variabile risposta <a id=studiare_esplicative_risposta> </a>

<div class="alert alert-danger fade in">
<strong>IMPORTANTE</strong>: Le analisi relative (anche) alla variabile risposta vanno effettuate utilizzando solo l'insieme di <em>training</em>. Utilizzare in questa fase anche gli insiemi di <em>validation</em> e/o di <em>test</em> può inficiare in modo più o meno grave le conclusioni che si traggono su di essi.
</div>

## 5.1 Relazione tra quantitative e risposta <a id=quantitative_risposta> </a>

In [ ]:
from msbd.grafici import grafico_dispersione_quantitative_risposta

print(inspect.getsource(grafico_dispersione_quantitative_risposta))

In [ ]:
plt.figure(figsize=(10, 10))

grafico_dispersione_quantitative_risposta(X.loc[idx_train], y.loc[idx_train], quantitative, 2)

plt.show()

## 5.2 Relazione tra qualitative e risposta <a id=qualitative_risposta> </a>

In [ ]:
from msbd.grafici import grafico_barre_qualitative_risposta

print(inspect.getsource(grafico_barre_qualitative_risposta))

In [ ]:
plt.figure(figsize=(10, 10))

grafico_barre_qualitative_risposta(X.loc[idx_train], y.loc[idx_train], qualitative, 2)

plt.show()

## 5.3 Relazione tra esplicative e risposta <a id=esplicative_risposta> </a>

In [ ]:
plt.figure(figsize=(8, 6))

sns.scatterplot(X.loc[idx_train, "Item_MRP"], y=y.loc[idx_train],
                hue=X.loc[idx_train, "Outlet_Type"],
                hue_order=["Supermarket Type3", "Supermarket Type2", "Supermarket Type1", "Grocery Store"],
                style=X.loc[idx_train, "Outlet_Location_Type"],
                style_order=["Tier 3", "Tier 2", "Tier 1"],
                size=X.loc[idx_train, "Outlet_Size"],
                size_order=["High", "Medium", "Small"])
plt.show()

# 6. Trasformare le variabili qualitative in dummy <a id=dummy> </a>

In [ ]:
from msbd.preprocessamento import OttenereDummy

print(inspect.getsource(OttenereDummy))

### Esercizio

Per come è definito, il metodo `fit()` della classe `OttenereDummy` crea le variabili dummy solo per salvarne i nomi. Ottenere lo stesso risultato senza utilizzare la funzione `get_dummies()` e senza creare le dummy in `fit()` (le dummy verranno create in `transform()`).

In [ ]:
print("X prima della creazione delle variabili dummy:")
display(X.head(2))

od = OttenereDummy(drop_first=True)

X = od.fit_transform(X)

print("\nX dopo la creazione delle variabili dummy:")
X.head(2)

### Esercizio

Perché abbiamo scelto `drop_first=True`?

# 7. Standardizzare i dati <a id=standardizzare> </a>

In [ ]:
from msbd.preprocessamento import Standardizzare

print(inspect.getsource(Standardizzare))

In [ ]:
print("X prima della standardizzazione:")
display(X.head(2))

std = Standardizzare()

X = std.fit_transform(X)

print("X dopo la standardizzazione:")
X.head(2)

### Esercizio

1. Completare il metodo `inverse_transform()` della classe `Standardizzare`;
2. Verificare il corretto funzionamento di `inverse_transform()` utilizzando `pytest`.

# 8. Pipeline di preprocessamento <a id=pipeline> </a>

In [ ]:
from sklearn.pipeline import Pipeline

## 8.1 Definire una `Pipeline` <a id=definire_pipeline> </a>

In [ ]:
preproc = Pipeline([
    ("riempire_na_item_weight", RiempireNAItemWeight()),
    ("riempire_na_outlet_size", RiempireNAOutletSize()),
    ("riempire_na_media", RiempireNAMedia()),
    ("eliminare_item_identifier", Eliminare(columns="Item_Identifier")),
    ("ottenere_dummy", OttenereDummy(drop_first=True)),
    ("standardizzare", Standardizzare()),
])

## 8.2 Preprocessare i dati attraverso la pipeline <a id=preprocessare_pipeline> </a>

In [ ]:
risposta = "Item_Outlet_Sales"
esplicative = sorted(col for col in dati.columns if col != risposta)

X, y = dati[esplicative].copy(), dati[risposta].copy()

In [ ]:
print("Dati grezzi:")
display(X.head(2))

X_preproc = preproc.fit_transform(X)

print("Dati preprocessati:")
X_preproc.head(2)

### Esercizio

Verificare che la pipeline costruita applichi tutte le trasformazioni viste in questo notebook analizzando `X_preproc`.

# 9. Salvare <a id=salvare> </a>

In [ ]:
OUTPUT_DIR = "output/07"

os.makedirs(OUTPUT_DIR, exist_ok=True) # se non esiste, crea la cartella finale e tutte le intermedie

## 9.1 Salvare i dati preprocessati <a id=salvare_dati> </a>

In [ ]:
X_preproc.loc[idx_train].to_pickle(OUTPUT_DIR + "/X_train.pkl")
X_preproc.loc[idx_val].to_pickle(OUTPUT_DIR + "/X_val.pkl")
X_preproc.loc[idx_test].to_pickle(OUTPUT_DIR + "/X_test.pkl")
y.loc[idx_train].to_pickle(OUTPUT_DIR + "/y_train.pkl")
y.loc[idx_val].to_pickle(OUTPUT_DIR + "/y_val.pkl")
y.loc[idx_test].to_pickle(OUTPUT_DIR + "/y_test.pkl")

## 9.2 Salvare la pipeline <a id=salvare_pipeline> </a>

In [ ]:
from sklearn.externals import joblib

joblib.dump(preproc, OUTPUT_DIR + "/preproc.joblib")